In [271]:
#STREAMS2TABLE
#written by Zachary J.J. Roper
#z.roper@vanderbilt.edu, zach@roper.io

#A script for converting raw event codes and timestamps into a trial-item dataset

In [272]:
#This notebook requires the following modules
import os
import numpy
import csv

In [273]:
#READ IN EVENT CODES
#The prepared file, out.csv, is a 2 column x session length (based on total number of trials) consistinting of event 
#codes (EVENTS[]) and their paired timestamps(TIMES[]). The file streams2text.m was used to generate these data 
#from the TDT data dump.

In [274]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)
pairedlist = list(zip(events,times))

In [275]:
events.count('2756') #catch trials

12

In [276]:
trial = 0
triallist = [None]*len(pairedlist)
eclist = [None]*len(pairedlist)
for i, data in enumerate(pairedlist):
    if pairedlist[i][0] == '2301':
        trial += 1
    triallist[i] = trial  
    eclist[i] = i
uberlist = list(zip(triallist,eclist,pairedlist))

In [277]:
#uberlist

In [278]:
'2810' in uberlist[74][2]

False

In [279]:
RT = [None]*len(pairedlist)
RTbeg = 0*len(pairedlist)
RTend = 0*len(pairedlist)
for i, trial in enumerate(uberlist):
    #print(i,trial)
    if '2651' in uberlist[i][2]:
        RTbeg = float(uberlist[i][2][1])
        focusTrial = uberlist[i][0]
        for p, case in enumerate(uberlist):
            if '2810' in uberlist[p][2] and uberlist[i][2] == focusTrial:
                RTend = float(uberlist[p][2][1])
    RT[i] = RTend - RTbeg

In [280]:
#RT

In [281]:
#The following provides indexed lists for each of the individual event codes of interest 

In [282]:
TargetOnset = []
FixOnset = []
Saccade = []
for i, x in enumerate(events):
    #print(i,x)
    if x == '2301':
        FixOnset.append(i)
    if x == '2651':
        TargetOnset.append(i)
    if x == '2810':
        Saccade.append(i)

In [283]:
#float(times[74])-float(times[70])

In [284]:
#Saccade

In [285]:
#for i, x in enumerate(Saccade):
#    print(Saccade[i],TargetOnset[i])

In [286]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [287]:
Trials = []
TimeTrials = []
for i, x in enumerate(FixOnset):
    if i < len(FixOnset)-1:
        a = FixOnset[i]
        b = FixOnset[i+1]-1
        Trials.append(events[a:b])
        TimeTrials.append(times[a:b])
    else:
        a = FixOnset[i]
        Trials.append(events[a:])
        TimeTrials.append(times[a:])

In [288]:
#print(TimeTrials[0])

In [289]:
path = os.getcwd()
csvfile = (path+"/table.csv")
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(Trials)

In [290]:
accuracy = [None]*len(Trials)
abort = [None]*len(Trials)
error = [None]*len(Trials)
catch = [None]*len(Trials)
correct = [None]*len(Trials)
tloc = [None]*len(Trials)
for i, trial in enumerate(Trials):
    if '2600' in trial:
        accuracy[i] = 1 #Correct Saccade (2600)
    else:
        accuracy[i] = 0  
    if '2651' in trial: #fixation was maintained sufficiently long enough for target onset (2651)
        abort[i] = 0
    else:
        abort[i] = 1
    if '887' in trial:
        error[i] = 1
    else:
        error[i] = 0
    if '2756' in trial:
        catch[i] = 1
    else:
        catch[i] = 0
    if '888' in trial:
        correct[i] = 1
    else:
        correct[i] = 0
    #if '5000' in trial:
    #    tloc[i] = 360
    if '5045' in trial:
        tloc[i] = 45
    elif '5090' in trial:
        tloc[i] = 90
    elif '5135' in trial:
        tloc[i] = 135
    elif '5180' in trial:
        tloc[i] = 180
    elif '5225' in trial:
        tloc[i] = 225
    elif '5270' in trial:
        tloc[i] = 270
    elif '5315' in trial:
        tloc[i] = 315
    else:
        tloc[i] = 0

In [291]:
new_dict = {}
for i, trial in enumerate(Trials):
    new_dict[i+1] = dict(zip(Trials[i],TimeTrials[i]))

In [292]:
RT = [0]*len(Trials)
focusTrial = 0
for i, trial in enumerate(Trials):
    if '2651' in trial and '2810' in trial: #and '2600' in trial:
        focusTrial = i + 1
        for p, case in enumerate(uberlist):
            if uberlist[p][0] == focusTrial:
                if '2810' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTend = float(uberlist[p][2][1])
                if '2651' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTbeg = float(uberlist[p][2][1])
            RT[i] = RTend - RTbeg

In [293]:
#RT

In [294]:
checksum = sum(error)+sum(abort)+sum(catch)

In [295]:
checksum

258

In [296]:
path = os.getcwd()
csvfile = (path+"/behaviorArray.csv")
trial = 0
triallist = [None]*len(abort)
with open(csvfile, "w") as output:
    for i,x in enumerate(abort):
        trial += 1
        triallist[i] = trial 
        print(i+1,abort[i],accuracy[i],error[i],correct[i],catch[i],RT[i],tloc[i])
    header = ['Trial', 'Abort', 'Accuracy', 'Error', 'Correct', 'Catch', 'RT', 'TarLoc']    
    dataset = list(zip(triallist,abort,accuracy,error,correct,catch,RT,tloc))
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(dataset)

1 0 0 0 0 0 0 225
2 1 0 0 0 0 0 45
3 0 1 0 1 0 241.99179999999978 135
4 1 0 0 0 0 0 225
5 0 0 1 0 0 198.0007000000005 180
6 0 1 0 1 0 198.0007000000005 180
7 0 1 0 1 0 901.9801000000007 135
8 0 1 0 1 0 198.0004000000008 315
9 1 0 0 0 0 0 90
10 0 1 0 1 0 165.96979999999894 90
11 0 1 0 1 0 161.99679999999353 315
12 0 1 0 1 0 154.0095999999976 180
13 0 1 0 1 0 158.02360000000044 315
14 0 1 0 1 0 198.0416999999943 0
15 1 0 0 0 0 0 270
16 0 0 1 0 0 176.0048999999999 45
17 0 1 0 1 0 198.0005999999994 270
18 0 1 0 1 0 154.0098999999973 0
19 0 1 0 1 0 201.97400000000198 0
20 0 1 0 1 0 207.9539999999979 45
21 0 0 1 0 0 178.0121999999974 45
22 1 0 0 0 0 0 270
23 0 1 0 1 0 420.00359999999637 270
24 1 0 0 0 0 0 315
25 0 0 1 0 0 239.98479999999108 270
26 0 1 0 1 0 221.96219999999448 0
27 1 0 0 0 0 0 45
28 0 1 0 1 0 154.00960000000487 135
29 0 1 0 1 0 201.97360000001208 135
30 1 0 0 0 0 0 315
31 0 0 1 0 0 180.01910000000498 225
32 0 1 0 1 0 187.9653999999864 90
33 0 1 0 1 0 199.96659999999974 315
34

520 0 0 0 1 0 396.0013999999501 270
521 0 0 1 0 0 420.00359999993816 45
522 0 0 0 1 0 209.96099999989383 315
523 1 0 0 0 0 0 0
524 0 1 0 1 0 202.0148999998346 45
525 1 0 0 0 0 0 180
526 0 1 0 1 0 195.99340000003576 270
527 1 0 0 0 0 0 270
528 0 1 0 1 0 313.9991000001319 45
529 0 1 0 1 0 200.00769999995828 45
530 0 1 0 1 0 171.99090000009164 270
531 0 1 0 1 0 266.03509999997914 270
532 0 1 0 1 0 207.9946999999229 90
533 0 1 0 1 0 381.99279999989085 270
534 0 1 0 1 0 193.98650000011548 0
535 0 1 0 1 0 203.9808999998495 135
536 0 1 0 1 0 185.9993999998551 315
537 1 0 0 0 0 0 90
538 0 1 0 1 0 165.96990000014193 90
539 0 1 0 1 0 178.01210000016727 0
540 0 0 1 0 0 201.93299999996088 270
541 0 1 0 1 0 231.99750000005588 180
542 0 0 1 0 0 229.99050000007264 90
543 1 0 0 0 0 0 225
544 0 0 0 1 0 255.91809999989346 45
545 0 1 0 1 0 229.9904999998398 45
546 0 1 0 1 0 228.02429999993183 0
547 1 0 0 0 0 0 270
548 0 1 0 1 0 253.9930999998469 225
549 0 1 0 1 0 236.0117999999784 0
550 0 1 0 1 0 282.009

In [297]:
sum(catch)

12